In [1]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from neo4j import GraphDatabase

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test1234"), encrypted=False)

In [3]:
def createGraphProjection(tx):
    return tx.run(
        """CALL gds.graph.create(
            'payment', 
            { Client: { label: 'Client', properties: { class: { property: 'class'} } } }, 
            { PAYS: { type: 'PAYS', orientation:'NATURAL'} }, 
            {
                nodeProperties: { },
                relationshipProperties: {amount: { property: 'amount', aggregation: 'NONE', defaultValue: 0.0 }}
            }
        )"""
    ).consume()

In [4]:
def runFastRP(tx): 
    return tx.run(
        """CALL gds.fastRP.mutate(
            'payment', 
            {
                embeddingDimension: 8,
                iterationWeights: [0.0, 0.20, 0.40, 0.60, 0.80, 1.00, 1.20, 1.40, 1.60, 1.80, 2.00],
                relationshipWeightProperty: 'amount',
                mutateProperty:'fastRP'
            }
            )"""
    ).consume()    

In [5]:
def trainNodeClassification(tx):
    return tx.run(
        """CALL gds.alpha.ml.nodeClassification.train('payment', {
            nodeLabels: ['Client'],
            modelName: 'fraud-model',
            featureProperties: ['fastRP'],
            targetProperty: 'class',
            randomSeed: 2,
            holdoutFraction: 0.3,
            validationFolds: 5,
            metrics: [ 'ACCURACY' ],
            params: [
                {penalty: 0.0625},
                {penalty: 0.5},
                {penalty: 1.0},
                {penalty: 4.0}
                ]
            }
            )"""
    ).single()['modelInfo']
    

In [6]:
def dropGraphProjection(tx):
    return tx.run("CALL gds.graph.drop('payment')").consume()

In [7]:
def dropModel(tx):
    return tx.run("CALL gds.beta.model.drop('fraud-model')").consume()

In [8]:
# Create projection 
with driver.session(database = "paysim") as session:
    result = session.read_transaction(createGraphProjection)
    print(result)

In [9]:
# Run FastRP embedding
with driver.session(database = "paysim") as session:
    result = session.write_transaction(runFastRP)
    print(result)

In [10]:
# Run Node Classification ML
with driver.session(database = "paysim") as session:
    result = session.write_transaction(trainNodeClassification)
    print(result)

{'name': 'fraud-model', 'metrics': {'ACCURACY': {'outerTrain': 0.91210225, 'test': 0.91359903, 'validation': [{'avg': 0.9121022560000001, 'min': 0.91164004, 'params': {'penalty': 0.0625}, 'max': 0.91229215}, {'avg': 0.9121022560000001, 'min': 0.91164004, 'params': {'penalty': 0.5}, 'max': 0.91229215}, {'avg': 0.912232678, 'min': 0.9119661, 'params': {'penalty': 1.0}, 'max': 0.9126182}, {'avg': 0.9123630779999999, 'min': 0.9119661, 'params': {'penalty': 4.0}, 'max': 0.9126182}], 'train': [{'avg': 0.91210225, 'min': 0.91205478, 'params': {'penalty': 0.0625}, 'max': 0.91221779}, {'avg': 0.91210225, 'min': 0.91205478, 'params': {'penalty': 0.5}, 'max': 0.91221779}, {'avg': 0.912265262, 'min': 0.91205478, 'params': {'penalty': 1.0}, 'max': 0.91246231}, {'avg': 0.9123630739999999, 'min': 0.9122993, 'params': {'penalty': 4.0}, 'max': 0.91246231}]}}, 'type': 'multiClassNodeLogisticRegression', 'bestParameters': {'penalty': 4.0}, 'classes': [0, 1]}


In [11]:
# Drop projection
with driver.session(database = "paysim") as session:
    result = session.read_transaction(dropGraphProjection)
    print(result)

In [12]:
# Drop Node Classification Model
with driver.session(database = "paysim") as session:
    result = session.read_transaction(dropModel)
    print(result)